In [ ]:
### Computer vision code for ball detection + support vector machine algorithm for predictions

# import the necessary packages

import itertools
from collections import deque
from imutils.video import VideoStream
import numpy as np
import cv2
import imutils
import time
import serial
import time


Stored_Positions = []     #ball postitions will be stored in this list
timer = 0                 # a parameter that helps in storing positions in other parts later

# define the lower and upper boundaries of the
# ball in the HSV color space, then initialize the
# list of tracked points, the frame counter,
# and the coordinate deltas

Lower1 = (0,121,128)
Upper1 = (179,255,255)

Lower= (0,68,109)
Upper= (179,255,255)
pts = deque(maxlen=32)

# User defined functions

def upper_Distance_from_axis(center_coordinate):
    x_coordinate = center_coordinate[0]
    z_coordinate = center_coordinate[1] 
    Coordinate = (x_coordinate,z_coordinate)
    return Coordinate

def bottom_Distance_from_axis(center_coordinate):
    y_coordinate = center_coordinate[1] 
    z_coordinate = center_coordinate[0]
    Coordinate = (z_coordinate,y_coordinate)
    return Coordinate


#video capture

vs = cv2.VideoCapture(0)    #USB camera at COM1 port 
VS = cv2.VideoCapture(1)    #USB camera at COM2 port


# allow the camera or video file to warm up
time.sleep(2.0)


# as long as the stored positions don't reach reach 5, the detection continues 

while len(Stored_Positions) <5:
    
    # grab the current frame
    
    ret,frame = vs.read()
    ret1,frame1 = VS.read()
    
    #resize and blurr the current frame
    
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    frame1 = imutils.resize(frame1, width=600)
    blurred1 = cv2.GaussianBlur(frame1, (11, 11), 0)
    hsv1 = cv2.cvtColor(blurred1, cv2.COLOR_BGR2HSV)
    
    # construct a mask for the color , then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    
    mask = cv2.inRange(hsv, Lower1,Upper1)
    mask = cv2.erode(mask, None, iterations=3)
    mask = cv2.dilate(mask, None, iterations=3)
    
    mask1 = cv2.inRange(hsv1, Lower,Upper)
    mask1 = cv2.erode(mask1, None, iterations=3)
    mask1 = cv2.dilate(mask1, None, iterations=3)
    
    
# find contours in the mask and initialize the current
    # (x, y) center of the ball
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    
    cnts1 = cv2.findContours(mask1.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts1 = imutils.grab_contours(cnts1)
    center1 = None
    
    # only proceed if at least one contour was found
    
    if len(cnts) > 0 and len(cnts1) > 0:
        
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        c1 = max(cnts1, key=cv2.contourArea)
        ((x1, y1), radius1) = cv2.minEnclosingCircle(c1)
        M1 = cv2.moments(c1)
        center1 = (int(M1["m10"] / M1["m00"]), int(M1["m01"] / M1["m00"]))
        
 
        
        
        # only proceed if the radius meets a minimum size
        
        if radius > 5 and radius1 > 5 :
            
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(frame, center, 1 , (0, 0, 255), -1)
           
            
            D = upper_Distance_from_axis(center)
            cv2.putText(frame,"Distance from axis", (center[0],center[1]+40), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(0, 255,0),1)
            cv2.putText(frame,"("+str(D[0])+","+str(D[1])+")", (center[0],center[1]+50), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(0, 255,0),1)
            
            cv2.circle(frame1, (int(x1), int(y1)), int(radius1),
                (0, 255, 255), 2)
            cv2.circle(frame1, center1, 1, (0, 0, 255), -1)
            
            
            D1 = bottom_Distance_from_axis(center1)
            cv2.putText(frame1,"Distance from axis", (center1[0],center1[1]+40), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(0, 255,0),1)
            cv2.putText(frame1,"("+str(D1[0])+","+str(D1[1])+")", (center1[0],center1[1]+50), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(0, 255,0),1)
           
            # start storing positions only if the current z-coordinate(depth)
            # exceeds a particular threshold(distance)
            
            if D1[0] >= (190): #and D1[0]>= (10):
                timer = 1
                if len(Stored_Positions)==0: 
                    BallCoordinate = (D[0],D1[1],D1[0])
                    Stored_Positions.append(BallCoordinate)
                else :
                    if D1[0]  > Stored_Positions[-1][-1]:
                        BallCoordinate = (D[0],D1[1],D1[0])
                        Stored_Positions.append(BallCoordinate)
                    else: pass
            
            # only store the current position if it is closer tothe goal than the previous one        
                
            else:
                if timer ==1:
                    if D1[0] > Stored_Positions[-1][-1]:
                        BallCoordinate = (D[0],D1[1],D1[0])
                        Stored_Positions.append(BallCoordinate)
                    else:
                        pass
            


# show the frame to our screen 

    cv2.imshow("upper camera", frame)
    cv2.imshow("bottom camera", frame1)
    key = cv2.waitKey(1) & 0xFF

    
    # if the 'q' key is pressed, stop the loop
    
    if key == ord("q")  :
        break
        
VS.release() 
vs.release()

# close all windows

cv2.destroyAllWindows()

#reshape the shots array
Stored_Positions = list(itertools.chain(*Stored_Positions))
Stored_Positions = np.array(Stored_Positions)#1
Stored_Positions =np.array(Stored_Positions)
Stored_Positions = Stored_Positions[np.newaxis,:]

#trigger the ML algorithm

result=svc.predict(Stored_Positions)



#transform the predicted label into a string
message = str(result[0])

#serial communication
serialcomm = serial.Serial('COM3', 9600)
#time.sleep(2)
serialcomm.write(message.encode())
serialcomm.close()
